In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms

# Set hyperparameters
batch_size = 64
input_size = 28  # MNIST images are 28x28 pixels
hidden_size = 128
num_layers = 2
num_classes = 10
num_epochs = 10
learning_rate = 0.001

# Load the MNIST dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)
trainset = MNIST(root="./data", train=True, download=True, transform=transform)
testset = MNIST(root="./data", train=False, download=True, transform=transform)

# Create data loaders
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

In [5]:
# Define the RNN model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out


# Initialize the model
model = RNN(input_size, hidden_size, num_layers, num_classes)

In [6]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
total_step = len(trainloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        images = images.reshape(-1, input_size, input_size)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(
                f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}"
            )

# Testing
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        images = images.reshape(-1, input_size, input_size)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

Epoch [1/10], Step [100/938], Loss: 1.0170
Epoch [1/10], Step [200/938], Loss: 0.6603
Epoch [1/10], Step [300/938], Loss: 0.7353
Epoch [1/10], Step [400/938], Loss: 0.7532
Epoch [1/10], Step [500/938], Loss: 0.4029
Epoch [1/10], Step [600/938], Loss: 0.3367
Epoch [1/10], Step [700/938], Loss: 0.3310
Epoch [1/10], Step [800/938], Loss: 0.2225
Epoch [1/10], Step [900/938], Loss: 0.4254
Epoch [2/10], Step [100/938], Loss: 0.1200
Epoch [2/10], Step [200/938], Loss: 0.2818
Epoch [2/10], Step [300/938], Loss: 0.1640
Epoch [2/10], Step [400/938], Loss: 0.2877
Epoch [2/10], Step [500/938], Loss: 0.2575
Epoch [2/10], Step [600/938], Loss: 0.0830
Epoch [2/10], Step [700/938], Loss: 0.3807
Epoch [2/10], Step [800/938], Loss: 0.0708
Epoch [2/10], Step [900/938], Loss: 0.1946
Epoch [3/10], Step [100/938], Loss: 0.2401
Epoch [3/10], Step [200/938], Loss: 0.1869
Epoch [3/10], Step [300/938], Loss: 0.2169
Epoch [3/10], Step [400/938], Loss: 0.1169
Epoch [3/10], Step [500/938], Loss: 0.0386
Epoch [3/10